In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
! pwd

The column data are the values for wavelength [nm], CD [mdeg], HT [V], and Abs. The wavelength was collected between 320 - 180 nm at 0.5 nm intervals (281 data points).

In [ ]:
def get_file_names(file_path):
    """
    Fetch the names of all files in the file path, and returns a list of all the .csv files.
    """ 
    file_list = os.listdir(file_path)
    file_names = sorted([file for file in file_list if file.endswith('.csv')])
    
    return file_names

In [ ]:
def get_cd_data(file):
    """
    Takes a .csv file, extracts the column data for Wavelength [nm], CD [mdeg], HT [V], and Abs into a DataFrame,
    and returns the Wavelength [nm] and CD [mdeg] values in the wavelength range 300 to 180 nm.
    
    This function was written for .csv files generated by a Jasco Spectra Manager software with data recorded on 
    a Jasco J-815 Spectropolarimeter set to scan at the wavelength range of 320 to 180 nm at 0.5 nm intervals.
    """
    df = pd.read_csv(file_path+file,
                     skiprows=19, 
                     skipfooter=45, 
                     names=['Wavelength [nm]', 'CD [mdeg]', 'HT [V]', 'Abs'], 
                     engine='python')
    xvalues = df['Wavelength [nm]'].values
    yvalues = df['CD [mdeg]'].values
    
    return xvalues, yvalues

In [ ]:
def get_label_name(file):
    """
    Takes a .csv file path and extracts the name of the test substrate after "TITLE," from the first line in the 
    file. The file name is in the format:
    "TITLE, [protein ratio]_[substrate ratio] [concentration of protein]mM [protein name]_[substrate name].csv"
    e.g. "TITLE, 5_1 10mM protein_substrate.csv"

    This function was written for .csv files generated by a Jasco Spectra Manager software with data recorded on 
    a Jasco J-815 Spectropolarimeter.
    """
    data_file = open(file_path+file, 'r')
    title_line = data_file.readline()
    data_file.close
    
    words = title_line.split(',')[1]
    label = words.split(':')[2].strip()
    
    return label

In [ ]:
def plot_cd(file_names, default=True, outfile):
    """
    Takes a list of .csv files and returns a CD plot of CD [mdeg] vs Wavelength [nm] between 300 to 180 nm for
    every file in the list and saves the figure as a .png, .svg, or .pdf.
    
    If default is set to True the y axis limits are contrained to -50 to 50 mdeg. Setting default to False uses 
    the appropriate y axis limits as determined by matplotlib.
    """
    # Set figure parameters
    fig, ax = plt.subplots(figsize=(16,9))
    
    # Set x axis coordinates from 180 to 300 nm.
    ax.set_xlim(180, 300)
    ax.set_xticks(np.arange(180, 301, step=10))
    ax.set_xticks(np.arange(180, 301, step=5), minor=True)
    
    # Constrain y axis limits from -50 to 50 mdeg if default is set to True
    if default:
        ax.set_ylim(-50, 50)
        ax.set_yticks(np.arange(-50, 51, step=10))
        ax.set_yticks(np.arange(-50, 51, step=5), minor=True)

    ax.spines['bottom'].set_position('zero') # Move x-axis to intersect at y=0
    ax.spines['right'].set_visible(False) # Eliminate 'right' axes
    ax.spines['top'].set_visible(False) # Eliminate 'top' axes

    ax.set_xlabel('Wavelength (nm)', fontsize=10, loc='right')
    ax.set_ylabel('CD (mdeg)', fontsize=10)

    ax.yaxis.grid()
        
    # Loop over the files in file_names and generate line for each one
    for file in file_names:
        xvalues, yvalues = get_cd_data(file)
        ax.plot(xvalues, yvalues, label=get_label_name(file))

    ax.legend()
    
    fig.savefig(outfile, dpi=300, transparent=True)
    
    return

In [ ]:
file_path = input('Enter file path to CD data: ')
outfile = input('Enter name of .png file to save: ')

plot_cd(get_file_names(file_path), outfile)